In [17]:
import pandas as pd
import sqlalchemy as sql
import psycopg2
from sql_functions import get_engine
from sql_functions import get_sql_config
from sql_functions import push_to_database

In [2]:
df_budget = pd.read_excel('data/brb_2022.xlsm')

In [3]:
df_budget.columns = df_budget.columns.str.replace(" ", "_")
df_budget.columns = df_budget.columns.str.lower()

In [4]:
df_budget.rename(columns = {'bezeichnung':'ep_txt', 'bezeichnung.1':'kapitel_txt', 'bezeichnung_fz' : 'fz_txt' }, inplace = True)

In [5]:
df_budget['ep'] = df_budget['ep'].astype(str)
df_budget['ep'] = df_budget['ep'].apply(lambda x:x.zfill(2))
df_budget['kapitel'] = df_budget['kapitel'].astype(str)
df_budget['kapitel'] = df_budget['kapitel'].str[-3:]
df_budget['titel'] = df_budget['titel'].astype(str)
df_budget['titel'] = df_budget['titel'].apply(lambda x:x.zfill(5))
df_budget['fz'] = df_budget['fz'].astype(str)
df_budget['fz'] = df_budget['fz'].apply(lambda x:x.zfill(3))


In [6]:
df_budget.dropna(subset=['ansatz_2022_in_eur'],inplace=True)
df_budget.ansatz_2022_in_eur = df_budget.ansatz_2022_in_eur.apply(lambda x: int(x))


In [7]:
df_budget["gruppe"] = df_budget["titel"].str[:3]
df_budget["counter"] = df_budget["titel"].str[3:]

In [8]:
df_budget = df_budget.drop(['tgr./mgr.','bezeichnung.2','titel','ansatz_2021_in_eur','ea'], axis=1)

In [9]:
df_budget['ansatz_2022_in_eur'] =df_budget[['ansatz_2022_in_eur', 'gruppe']].apply(lambda x:-x[0] if int(x[1]) > 400 else x[0], axis=1)

In [10]:
df_budget.insert(len(df_budget.columns),"jahr", 2022)
df_budget.insert(len(df_budget.columns),"state_id", 4)

In [11]:
df_budget

,ep,ep_txt,kapitel,kapitel_txt,fz,fz_txt,zweckbestimmung,ansatz_2022_in_eur,gruppe,counter,jahr,state_id
0,01,Landtag,010,Landtag Brandenburg,011,Politische Führung,"Gebühren, sonstige Entgelte",1000,111,10,2022,4
1,01,Landtag,010,Landtag Brandenburg,011,Politische Führung,"Geldstrafen, Geldbußen und Zwangsgelder (einsc...",1000,112,10,2022,4
2,01,Landtag,010,Landtag Brandenburg,011,Politische Führung,Sonstige Verwaltungseinnahmen,1000,119,10,2022,4
3,01,Landtag,010,Landtag Brandenburg,011,Politische Führung,Einnahmen aus Veröffentlichungen,2000,119,20,2022,4
4,01,Landtag,010,Landtag Brandenburg,012,Innere Verwaltung,Umsatzsteuererstattung des Finanzamtes,0,119,22,2022,4
...,...,...,...,...,...,...,...,...,...,...,...,...
5988,20,Allgemeine Finanzverwaltung,710,Versorgung,813,Sondervermögen,"Zuführung an das Sondervermögen ""Versorgungsfo...",0,919,35,2022,4
5989,20,Allgemeine Finanzverwaltung,710,Versorgung,018,Versorgung einschließlich Beihilfen für Versor...,Erstattungen von Versorgungsbezügen an den Bund,-600000,631,60,2022,4
5990,20,Allgemeine Finanzverwaltung,710,Versorgung,018,Versorgung einschließlich Beihilfen für Versor...,Erstattungen von Versorgungsbezügen an die Länder,-3360000,632,60,2022,4
5991,20,Allgemeine Finanzverwaltung,710,Versorgung,018,Versorgung einschließlich Beihilfen für Versor...,Erstattungen von Versorgungsbezügen an Gemeind...,-10000,633,60,2022,4


In [12]:
df_fz = df_budget[['fz','fz_txt']]
df_fz.drop_duplicates(inplace = True)

/var/folders/23/rcv81wbj68b4cdb5ycs5jp180000gn/T/ipykernel_1962/3190022759.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fz.drop_duplicates(inplace = True)


In [13]:
df_ep_txt = df_budget[['ep','ep_txt','jahr','state_id']]
df_ep_txt.drop_duplicates(inplace=True)

/var/folders/23/rcv81wbj68b4cdb5ycs5jp180000gn/T/ipykernel_1962/389507899.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ep_txt.drop_duplicates(inplace=True)


In [14]:
df_kapitel_txt = df_budget[['ep','kapitel','kapitel_txt','jahr','state_id']]
df_kapitel_txt.drop_duplicates(inplace=True)

/var/folders/23/rcv81wbj68b4cdb5ycs5jp180000gn/T/ipykernel_1962/3388781557.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kapitel_txt.drop_duplicates(inplace=True)


upload bundeslaender brb

In [16]:
df_fz.rename(columns={"fz":"fkz", "fz_txt":"fkz_txt"}, inplace=True)

/var/folders/23/rcv81wbj68b4cdb5ycs5jp180000gn/T/ipykernel_1962/2915164764.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fz.rename(columns={"fz":"fkz", "fz_txt":"fkz_txt"}, inplace=True)


In [18]:
get_sql_config()
engine = get_engine()


In [19]:
#push_to_database(df_fz, "funktionsbezeichnungen", engine, "capstone_public_budgeting")

The funktionsbezeichnungen table was imported successfully.


In [23]:
df_ep_txt.rename(columns={"jahr":"year"}, inplace=True)

/var/folders/23/rcv81wbj68b4cdb5ycs5jp180000gn/T/ipykernel_1962/181164097.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ep_txt.rename(columns={"jahr":"year"}, inplace=True)


In [25]:
#push_to_database(df_ep_txt, "einzelplaene", engine, "capstone_public_budgeting")

The einzelplaene table was imported successfully.


In [34]:
df_ep_txt

,ep,ep_txt,year,state_id
0,01,Landtag,2022,4
215,02,"Ministerpräsidentin, Ministerpräsident und Sta...",2022,4
389,03,Ministerium des Innern und für Kommunales,2022,4
963,04,Ministerium der Justiz,2022,4
1699,05,"Ministerium für Bildung, Jugend und Sport",2022,4
2191,06,"Ministerium für Wissenschaft, Forschung und Ku...",2022,4
2630,07,"Ministerium für Soziales, Gesundheit, Integrat...",2022,4
3324,08,"Ministerium für Wirtschaft, Arbeit und Energie",2022,4
3843,10,"Ministerium für Landwirtschaft, Umwelt und Kli...",2022,4
4728,11,Ministerium für Infrastruktur und Landesplanung,2022,4


In [27]:
df_kapitel_txt.rename(columns={"jahr":"year"}, inplace=True)

/var/folders/23/rcv81wbj68b4cdb5ycs5jp180000gn/T/ipykernel_1962/3686903904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kapitel_txt.rename(columns={"jahr":"year"}, inplace=True)


In [28]:
#push_to_database(df_kapitel_txt, "kapitel", engine, "capstone_public_budgeting")

The kapitel table was imported successfully.


In [32]:
#df_budget = df_budget.drop(['ep_txt', 'kapitel_txt', 'fz_txt'], axis=1)
df_budget.rename(columns={'ansatz_2022_in_eur':'amount', 'fz':'fkz', 'jahr':'year'}, inplace=True)
df_budget

,ep,kapitel,fkz,zweckbestimmung,amount,gruppe,counter,year,state_id
0,01,010,011,"Gebühren, sonstige Entgelte",1000,111,10,2022,4
1,01,010,011,"Geldstrafen, Geldbußen und Zwangsgelder (einsc...",1000,112,10,2022,4
2,01,010,011,Sonstige Verwaltungseinnahmen,1000,119,10,2022,4
3,01,010,011,Einnahmen aus Veröffentlichungen,2000,119,20,2022,4
4,01,010,012,Umsatzsteuererstattung des Finanzamtes,0,119,22,2022,4
...,...,...,...,...,...,...,...,...,...
5988,20,710,813,"Zuführung an das Sondervermögen ""Versorgungsfo...",0,919,35,2022,4
5989,20,710,018,Erstattungen von Versorgungsbezügen an den Bund,-600000,631,60,2022,4
5990,20,710,018,Erstattungen von Versorgungsbezügen an die Länder,-3360000,632,60,2022,4
5991,20,710,018,Erstattungen von Versorgungsbezügen an Gemeind...,-10000,633,60,2022,4


In [33]:
#push_to_database(df_budget, "budget_all", engine, "capstone_public_budgeting")

The budget_all table was imported successfully.
